In [1]:
from pyraf import iraf, irafpar
from pyraf.irafpar import IrafParList
import math
import numpy as np
import os
iraf.stsdas(_doprint=0)
iraf.analysis(_doprint=0)
iraf.isophote(_doprint=0)



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |
      |                   STSDAS Version 3.18.3                    |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+


In [ ]:
def init(target_name):

    input_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/optical/'
    uparm_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/uparm/'
    coord_file = ('/Users/orion/phd_research/CSS2/ap_phot_new/coord/'+target_name)
    output_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/optical/'
    input_image = input_dir+target_name+'_new.fits'

    iraf.flpr()
    iraf.noao(_doprint=0)
    iraf.digiphot(_doprint=0)
    iraf.apphot(_doprint=0)
    iraf.stsdas(_doprint=0)
    iraf.analysis(_doprint=0)
    iraf.isophote(_doprint=0)
    iraf.datapars.unlearn() 
    iraf.fitskyp.unlearn()
    iraf.fitsky.unlearn()
    iraf.tprint.unlearn()
    iraf.imgets.unlearn()
    iraf.mskregions.unlearn()
    iraf.fixpix.unlearn()
    iraf.controlpar.unlearn()
    iraf.geompar.unlearn()
    iraf.ellipse.unlearn()
    sma = 0.0
    intens = 0.0
    tflux = 0.0
    npix = 0.0
    ellip = 0.0
    pa = 0.0
    r_ba = 0.0
    magzp = 0.0
    msky, stdev = 0.0,0.0
    newX0, newY0 = 0., 0.
    photflam = 0.0
    st_mag, ab_mag = 0., 0.



In [ ]:
def centralpix(input_image, coord_file):

    #Converting RA and DEC in wcs to pixels
    iraf.wcsctran.setParam('input', coord_file)
    iraf.wcsctran.setParam('image', input_image)
    iraf.wcsctran.setParam('inwcs', 'world')
    iraf.wcsctran.setParam('outwcs', 'logical')
    iraf.wcsctran.setParam('columns', '1 2')
    iraf.wcsctran.setParam('units', 'd')
    xy = iraf.wcsctran(Stdout=1, mode='h')
    print xy
    xy = xy[3].split()  #Extracting only the x and y pixel position from the output string of wcsctran
    x0 = round(float(xy[0]), 1)
    y0 = round(float(xy[1]),1)
    print( 'x0 = ', x0)
    print( 'y0 = ', y0)
    
    return x0, y0

In [ ]:
def mask(input_dir, target_name, uparm_dir):

    #Creating mask regions using the .reg files of the respective .fits using the mskregions task

    iraf.mskregions.setParam('regions', input_dir+target_name+'_mask.reg')
    #iraf.mskregions.setParam('regions', '1203_companion_mask.reg')
    iraf.mskregions.setParam('masks', input_dir+target_name+'_mask.fits.pl')
    iraf.mskregions.setParam('refimage', input_dir+target_name+'_new.fits')
    iraf.mskregions.setParam('append', 'no')
    iraf.mskregions.setParam('verbose', 'no')
    iraf.mskregions.saveParList(filename = uparm_dir+'mskregions.par')
    iraf.mskregions(mode='l')
    print('Masking done')

In [ ]:
def runEllipse(x0, y0, magzp):
    print('Running ellipse first stage...')
    
    iraf.unlearn(iraf.geompar)
    iraf.unlearn(iraf.controlpar)
    iraf.unlearn(iraf.magpar)
    iraf.unlearn(iraf.ellipse)
    iraf.flprcache()
    iraf.ellipse.x0=x0
    iraf.ellipse.y0=y0
    iraf.ellipse.step=0.1
    iraf.ellipse.linear='no' 
    #iraf.ellipse.wander=3
    iraf.ellipse.maxgerr=1.       #fix maxgerr=1 for all bands    
    iraf.ellipse.mag0=magzp
    iraf.ellipse.input=input_dir+target_name+'_new.fits'
    iraf.ellipse.output=output_dir+target_name+'_isofit.tab'  
    iraf.ellipse.region = 'no'
    iraf.ellipse.interactive = 'no'
    iraf.ellipse.verbose = 'no'
    iraf.ellipse()   

In [ ]:
def zeropt(imput_image):

    #Read photometric zero-point from image header
    iraf.images.imutil.imgets(input_image, 'PHOTZPT')
    magzp = iraf.images.imutil.imgets.value              # magzp = photzpt i.e. zero-point of STmag scale (-21.1)
    iraf.images.imutil.imgets(input_image, 'PHOTFLAM')
    photflam = iraf.images.imutil.imgets.value
    iraf.images.imutil.imgets(input_image, 'PHOTPLAM')   #for ab_mag computation
    photplam = iraf.images.imutil.imgets.value
    print('photflam= ', photflam , '\n', 'photplam= ', photplam)

    return magzp, photflam, photplam

In [ ]:
def sky_estim(target_name, input_dir, output_dir, uparm_dir, coord_file, input_image):
    for line in reversed(open(output_dir+target_name+'_isofit.dat').readlines()):
    #for line in reversed(open(output_dir+target_name+'_companion_isofit.dat').readlines()):
        print line
        line = line.strip().split()
        if line[8].strip() !='INDEF' and line[9].strip()!='INDEF':    
            sma = float(line[0].strip())
            break
    for line in reversed(open(output_dir+target_name+'_isofit.dat').readlines()):
    #for line in reversed(open(output_dir+target_name+'_companion_isofit.dat').readlines()):
        line = line.strip().split()
        if float(line[1].strip()) >= (msky+1.*stdev):    #line[1] is intensity
            maxsma = float(line[0].strip())
        print('maxsma', maxsma)
        break
    
    #Parameters: scale - reduction-params (final_pixfrac, final_scale) -
    #                                   so check for each target (ds9 region analysis)
    #fwhmpsf - filter dependent, WFC3 instrument handbook :   op = 0.07 arcsec, uv = 


    #Setting datapars for fitsky
    iraf.datapars.setParam('scale', '0.07')
    iraf.datapars.setParam('fwhmpsf', '0.07')    
    #iraf.datapars.setParam('sigma', skysig)
    #iraf.datapars.setParam('itime', '1.3')
    iraf.datapars.saveParList(filename = uparm_dir+'fitsky_data.par')
        
    #setting fitskypars for fitsky
    iraf.fitskyp.setParam('salgori', 'mode')
    iraf.fitskyp.setParam('annulus', sma*0.04+50.*0.04)   #annulus in scale units, so multiplied by "scale" above
    iraf.fitskyp.setParam('dannulus', 20*0.04)
    #iraf.fitskyp.setParam('skyval', skyval)
    iraf.fitskyp.saveParList(filename = uparm_dir+'fitsky_fitsky.par')
    
    #Setting fitsky
    iraf.fitsky.setParam('image', input_dir+target_name+'_new.fits')
    iraf.fitsky.setParam('coords', coord_file)
    iraf.fitsky.setParam('output', output_dir+target_name+'.sky')
    iraf.fitsky.setParam('datapars', uparm_dir+'fitsky_data.par')
    iraf.fitsky.setParam('fitskypars', uparm_dir+'fitsky_fitsky.par')
    iraf.fitsky.setParam('interactive', 'no')
    iraf.fitsky.setParam('verbose', 'no')
    iraf.fitsky.setParam('verify', 'no')
    
    print iraf.fitsky.getParam("fitsky.image")
    iraf.fitsky(mode='h', Stdout=1)     #Executing fitsky task
    
    
    sky_values = []
    #Reading the output of fitsky task
    f = open(output_dir+target_name+'.sky', 'r')
    for line in f:
        if '#' not in line:
            temp = line.strip().split()
            sky_values.append(temp)
    f.close()        
    msky = float(sky_values[1][0])
    stdev = float(sky_values[1][1])
    print( "msky = ", msky)
    print( "stdev = ", stdev)
    
    return msky, stdev



In [ ]:
#Using msky+1sigma as the cut-off for the max sma to consider for ellipse isophote 
#and also the one that does not have INDEF in the ellip_err and pa_err and grad and grad_err column of output table.

#For those cases where auto-selection of 1sigma cutoff is not sensible (scattered emission cases), choose manual "new_sma"


#columns='SMA, intens, int_err, rms, ellip, ellip_err, 
#pa, pa_err, grad, grad_err,rsma, mag, mag_lerr,
#tflux_e, npix_e, X0, Y0, ndata'
 
def aperture(target_name, output_dir, msky, stdev):
    for line in reversed(open(output_dir+target_name+'_isofit.dat').readlines()):
    #for line in reversed(open(output_dir+target_name+'_companion_isofit.dat').readlines()):
        line = line.strip()
        line = line.split()
        if float(line[1].strip()) >= (msky+1.*stdev) and line[9].strip()!='INDEF':       #line[1] is intensity
            newX0 = float(line[15])      #column 15,16 - we designated these columns to x0, y0 in "tdump"
            newY0 = float(line[16])
            sma = float(line[0].strip())
            #sma = 178                        # manually selected (pixels)
            intens = float(line[1].strip())
            tflux = float(line[13].strip()) #column 13,14 - we designated these columns to tflux_e, npix_e 
            npix = float(line[14].strip())
            ellip = float(line[4].strip())
            pa = float(line[6].strip())
            r_ba = 1 - ellip
            print( 'new_sma= ', sma)
            break       
    print( "1sig_aperture params: x0=",newX0, "y0=",newY0, "ellip=", ellip, "pa=", pa)
    
    
    
    iraf.display.setParam('bpdispl', 'overlay')
    iraf.display.setParam('overlay',  input_dir+target_name+'.fits.pl')

    #output region file with ellipse and text
    region_file = open(output_dir+target_name+'.reg','w')
    #region_file = open(output_dir+target_name+'_companion.reg','w')
    region_file.write("# Region file format: DS9 version 6.2\n")
    region_file.write('#global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 image\n')
    region_file.write('ellipse(' + str(newX0) + ',' + str(newY0)+ ',' + str(sma) + ','+str(round(sma*r_ba,2)) + ','+ str(90.+pa) +  ')' + '\n')
    region_file.close()

    
    return npix


In [ ]:
def phot_calc(eer, npix, msky, magzp, photflam, photplam):

    #Subtracting sky background    
    tflux_bgsub = tflux - npix*msky
    st_mag = round((-2.5*math.log10(tflux_bgsub) + (2.5*math.log10(eer)) + float(magzp) - 2.5*math.log10(float(photflam))), 6)
    ab_mag = round((st_mag - 5*math.log10(float(photplam)) + float(18.69)), 6)
     
    #poisson noise (shot noise)
    n_poisson = math.sqrt(tflux)

    #error in sky
    n_sky = math.sqrt(stdev*npix)
    
    #total error
    n_tot = math.sqrt(n_poisson**2 + n_sky**2)    #add noise in quadrature
    tot_mag_err = round(2.5*math.log10(1.+n_tot/tflux), 2)

    print( "Target values:")
    print( "bkgd_sub_flux= ", tflux_bgsub, "total_valid_pixels= ", npix, "STmag= ", st_mag, "ABmag= ", ab_mag)
    print( "ABmag_err= ", tot_mag_err)

In [ ]:
###################    optical band

In [ ]:
targets = ['0258+35_f621m','1014+392_f845m','1025+390_f763m', '1037+30_f621m', '1128+455_f763m',
           '1201+394_f845m', '1203+645_f763m', '1221-423_f689m', '1445+410_f689m']

#x0,y0 --> "physical" co-ords, enter manually
x0 =  [489, 56, 203, 186, 203, 92, 185, 221, 140]          
y0 = [370, 48, 178, 153, 167, 94, 156, 171, 127]


#target_companions= ['1201+394_companion', '1203+645_companion']
# x0, y0

In [ ]:
for i in range(len(targets)):

    print(targets[i])
    init(targets[i])
    #x, y = centralpix
    x, y = x0[i], y0[i]
    print( 'x0 = ', x)
    print( 'y0 = ', y)
    
    mask(input_dir, targets[i], uparm_dir)
    
    #Ellipse run
    runEllipse(x0, y0, magzp) 

    iraf.tdump.setParam('table',output_dir+target_name+'_isofit.tab')
    #iraf.tdump.setParam('table',output_dir+target_name+'_companion_isofit.tab')
    iraf.tdump.setParam('cdfile','null')
    iraf.tdump.setParam('pfile','null')
    iraf.tdump.setParam('datafil', output_dir+target_name+'_isofit.dat')
    iraf.tdump.setParam('columns', 'SMA, intens, int_err, rms, ellip, ellip_err, pa, pa_err, grad, grad_err, rsma, mag, mag_lerr, tflux_e, npix_e, X0, Y0, ndata ')
    #selecting 18 columns from .tab file - and only these are copied to the .dat file 
    iraf.tdump.setParam('pwidth', 300)
    iraf.tdump.saveParList(uparm_dir+'tdump.par')
    iraf.tdump(mode='h')
    
    mag_zp, photFLAM, photPLAM = zeropt(image)
    mean_sky, std_sky = sky_estim(targets[i], input_dir, output_dir, uparm_dir, coord_file, input_image)
    total_pixels = aperture(targets[i], output_dir, mean_sky, std_sky)
    
    #Reading Encircled Energy values at given aperture size for the given filter
    #read FILTER keyword from header maybe
    #eer = []
    #for line in reversed(open(uvis2eef.dat').readlines()):
    #    line = line.split()
    #    if float(line[1].strip()) == 'F621M':       
    #       eer = float(line[0].strip())
    #      print 'EE(r)= ', eer
    #    break

    
    eer = 0.951      #manually enter from STScI .dat file 
    
    
    phot_calc(eer, total_pixels, mean_sky, mag_zp, photFLAM, photPLAM)
    



0258+35_f621m

x0 =  513

y0 =  376

Masking done

photflam=  3.9717599E-19 

photplam=  6218.80005

maxsma 547.6373

msky =  0.0618

stdev =  0.03629

new_sma=  309.127

1sig_aperture params: x0= 494.174 y0= 373.759 ellip= 0.16199 pa= -42.959

Target values:

bkgd_sub_flux=  56522.851 total_valid_pixels=  251577.0 STmag=  13.0219 ABmag=  12.743

ABmag_err= 0.0


1014+392_f845m

x0 =  56

y0 =  48

photflam=  4.5176800E-19 

photplam=  8440.550299999999

Running ellipse first stage...

maxsma 19.487


msky =  0.00368

stdev =  0.00853

new_sma=  17.71561

1sig_aperture params: X0= 56.39 Y0= 47.28 ellip= 0.3915 pa= 85.8744


Target values:

bkgd_sub_flux=  17.7300 total_valid_pixels=  593.0 STmag=  21.60 ABmag=  20.6648

ABmag_err= 0.24



1025+390_f763m

x0 =  203

y0 =  178

photflam=  3.7867899E-19 

photplam=  7614.30005

Running ellipse first stage...

maxsma 158.631

msky =  0.00215

stdev =  0.00532

new_sma=  81.40277

1sig_aperture params: newX0= 194.683 newY0= 182.802 ellip= 0.50823 pa= -84.84199

Target values:

bkgd_sub_flux=  231.214 total_valid_pixels=  10243.0 STmag=  19.01009 ABmag=  18.2919

ABmag_err= 0.07



1037+30_f621m

x0 =  186

y0 =  153

photflam=  3.9717599E-19 

photplam=  6.2188999E+03

Running ellipse first stage...

msky =  0.00355

stdev =  0.03722

new_sma=  89.543

1sig_aperture params: x0= 166.266 y0= 156.555 ellip= 0.5119 pa= -86.656

Target values:

bkgd_sub_flux=  2592.452 total_valid_pixels=  12279.0 STmag=  16.3682 ABmag=  16.08969

ABmag_err= 0.02


1128+455_f763m

x0 =  203

y0 =  167

photflam=  3.7867899E-19 

photplam=  7614.30005

Running ellipse first stage...

maxsma 55.599

msky =  0.00117

stdev =  0.00563

new_sma=  34.522

1sig_aperture params: x0= 207.101 y0= 165.579 ellip= 0.49322 pa= -37.119

Target values:

bkgd_sub_flux=  71.1494 total_valid_pixels=  1897.0 STmag=  20.2897 ABmag=  19.5715

ABmag_err= 0.13


1201+394_f845m

x0 =  92

y0 =  94

Masking done

photflam=  4.5176800E-19 

photplam=  8440.550299999999

Running ellipse first stage...

maxsma 55.599

msky =  0.0051

stdev =  0.00978

new_sma=  31.384

1sig_aperture params: x0= 91.048 y0= 96.891 ellip= 0.5872 pa= -32.623


Target values:

bkgd_sub_flux=  41.8598 total_valid_pixels=  1281.0 STmag=  20.665 ABmag=  19.723

ABmag_err= 0.16



1203+645_f763m

x0 =  185

y0 =  156

photflam=  3.7867899E-19 

photplam=  7614.30005

maxsma 41.7724

msky =  0.001899

stdev =  0.005859

new_sma=  25.937

1sig_aperture params: x0= 184.194 y0= 153.201 ellip= 0.3326 pa= 4.849

Target values:

bkgd_sub_flux=  33.7875 total_valid_pixels=  1411.0 STmag=  21.082 ABmag=  20.364

ABmag_err= 0.18



1221-423_f689m


x0 =  207

y0 =  163

photflam=  3.6718599E-19 

photplam=  6876.6499

maxsma 23.579

msky =  0.00629

stdev =  0.01619

new_sma=  130

1sig_aperture params: x0= 218.408 y0= 168.815 ellip= 0.1426 pa= -103.954

Target values:

bkgd_sub_flux=  394.611 total_valid_pixels=  1247.0 STmag=  18.450 ABmag=  17.9459

ABmag_err= 0.06


1445+410_f689m

x0 =  140

y0 =  126

photflam=  3.6718599E-19 

photplam=  6876.6499

Running ellipse first stage...

maxsma 74.00252

msky =  0.00734

stdev =  0.0147

new_sma=  37.97498

1sig_aperture params: x0= 140.836 y0= 129.1007 ellip= 0.356622 pa= -61.59815

Target values:

bkgd_sub_flux=  240.66 total_valid_pixels=  2917.0 STmag=  19.000084 ABmag=  18.503199

ABmag_err= 0.07


In [ ]:
###################    uv band

In [ ]:
targets = ['1025+390_f336w','1037+30_f225w','1128+455_f336w','1201+394_f336w', '1203+645_f336w', '1221-423_f275w']

#x0,y0 --> "physical" co-ords, enter manually
x0 =  [297, 71, 217, 178, 194, 225]          
y0 = [223, 56, 197, 174, 143, 216]

In [ ]:
def init(target_name):

    input_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/uv/'
    uparm_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/uparm/'
    coord_file = ('/Users/orion/phd_research/CSS2/ap_phot_new/coord/'+target_name)
    output_dir = '/Users/orion/phd_research/CSS2/ap_phot_new/uv/'
    input_image = input_dir+target_name+'_new.fits'

    iraf.flpr()
    iraf.noao(_doprint=0)
    iraf.digiphot(_doprint=0)
    iraf.apphot(_doprint=0)
    iraf.stsdas(_doprint=0)
    iraf.analysis(_doprint=0)
    iraf.isophote(_doprint=0)
    iraf.datapars.unlearn() 
    iraf.fitskyp.unlearn()
    iraf.fitsky.unlearn()
    iraf.tprint.unlearn()
    iraf.imgets.unlearn()
    iraf.mskregions.unlearn()
    iraf.fixpix.unlearn()
    iraf.controlpar.unlearn()
    iraf.geompar.unlearn()
    iraf.ellipse.unlearn()
    sma = 0.0
    intens = 0.0
    tflux = 0.0
    npix = 0.0
    ellip = 0.0
    pa = 0.0
    r_ba = 0.0
    magzp = 0.0
    msky, stdev = 0.0,0.0
    newX0, newY0 = 0., 0.
    photflam = 0.0
    st_mag, ab_mag = 0., 0.
    

In [ ]:
for i in range(len(targets)):

    print(targets[i])
    
    init2(targets[i])
    #x, y = centralpix
    x, y = x0[i], y0[i]
    print( 'x0 = ', x)
    print( 'y0 = ', y)
    
    mask(input_dir, targets[i], uparm_dir)
    
    #Ellipse run
    runEllipse(x0, y0, magzp) 

    iraf.tdump.setParam('table',output_dir+target_name+'_isofit.tab')
    #iraf.tdump.setParam('table',output_dir+target_name+'_companion_isofit.tab')
    iraf.tdump.setParam('cdfile','null')
    iraf.tdump.setParam('pfile','null')
    iraf.tdump.setParam('datafil', output_dir+target_name+'_isofit.dat')
    iraf.tdump.setParam('columns', 'SMA, intens, int_err, rms, ellip, ellip_err, pa, pa_err, grad, grad_err, rsma, mag, mag_lerr, tflux_e, npix_e, X0, Y0, ndata ')
    #selecting 18 columns from .tab file - and only these are copied to the .dat file 
    iraf.tdump.setParam('pwidth', 300)
    iraf.tdump.saveParList(uparm_dir+'tdump.par')
    iraf.tdump(mode='h')
    
    mag_zp, photFLAM, photPLAM = zeropt(image)
    mean_sky, std_sky = sky_estim(targets[i], input_dir, output_dir, uparm_dir, coord_file, input_image)
    total_pixels = aperture(targets[i], output_dir, mean_sky, std_sky)
    
    eer = 0.951      #manually enter from STScI .dat file 
    
    phot_calc(eer, total_pixels, mean_sky, mag_zp, photFLAM, photPLAM)
    

1025+390_f336w

x0 =  297

y0 =  223

photflam=  1.26612E-18 

photplam=  3354.84995

Running ellipse first stage...

maxsma 19.48717

msky =  0.000120  

stdev =  0.003076

Target values: 

bkgd_sub_flux=  5.731 total_valid_pixels=  2820.0 STmag=  21.651 ABmag=  22.713

ABmag_err= 0.39



1037+30_f225w

x0 =  71

y0 =  57

photflam=  4.5188302E-18 

photplam=  2.3595000E+03

Running ellipse first stage...

maxsma 12.1

msky =  0.00139

stdev =  0.02504

Target values:

bkgd_sub_flux=  39.7622 total_valid_pixels=  2327.0 STmag=  18.154 ABmag=  19.980

ABmag_err= 0.22



1128+455_f336w

x0 = 217

y0 = 197

photflam=  1.26612E-18 

photplam=  3354.84995

Running ellipse first stage...

maxsma 13.31

msky =  0.000115

stdev =  0.00291

Target values:

bkgd_sub_flux=  4.413 total_valid_pixels=  1465.0 STmag=  21.922 ABmag=  22.984

ABmag_err= 0.67



1201+394_f336w


x0 = 178

y0 = 173

photflam=  1.26612E-18 

photplam=  3354.84995

Running ellipse first stage...

maxsma 11.0

msky =  0.000817

stdev =  0.004416

Target values:

bkgd_sub_flux=  0.98105 total_valid_pixels=  383.0 STmag=  23.621 ABmag=  24.682

ABmag_err= 1.06 




1203+645_f336w

x0 = 194

y0 = 143

photflam=  1.26612E-18 

photplam=  3354.84995

Running ellipse first stage...

maxsma 13.31

msky =  0.0003474

stdev =  0.00414

Target values:

bkgd_sub_flux=  5.5989 total_valid_pixels=  3874.0 STmag=  21.7438 ABmag=  22.8054

ABmag_err= 0.81



1221-423_f275w


x0 =  225

y0 =  216

photflam=  3.1858499E-18 

photplam=  2707.19995

Running ellipse first stage...

maxsma 12.1

msky =  0.001137

stdev =  0.006007

Target values:

bkgd_sub_flux=  46.90399 total_valid_pixels=  94813 STmag=  18.4639 ABmag=  19.9913

ABmag_err= 0.43

